In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [33]:
# Hyperparameters
num_epochs = 30
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
train_dataset = torchvision.datasets.MNIST(root='/data', train=True, transform=transforms.ToTensor(), download=True)

100.1%

Extracting /data\MNIST\raw\train-images-idx3-ubyte.gz


113.5%

Extracting /data\MNIST\raw\train-labels-idx1-ubyte.gz


100.4%

Extracting /data\MNIST\raw\t10k-images-idx3-ubyte.gz


180.4%

Extracting /data\MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [7]:
test_dataset = torchvision.datasets.MNIST(root='/data', train=False, transform=transforms.ToTensor())

In [8]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [10]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [11]:
# Convolutional Neural Network
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        
        return out

In [12]:
model = ConvNet(num_classes).to(device)

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [34]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/30], Step [100/600], Loss: 0.0000
Epoch [1/30], Step [200/600], Loss: 0.0000
Epoch [1/30], Step [300/600], Loss: 0.0005
Epoch [1/30], Step [400/600], Loss: 0.0023
Epoch [1/30], Step [500/600], Loss: 0.0000
Epoch [1/30], Step [600/600], Loss: 0.0001
Epoch [2/30], Step [100/600], Loss: 0.0000
Epoch [2/30], Step [200/600], Loss: 0.0000
Epoch [2/30], Step [300/600], Loss: 0.0000
Epoch [2/30], Step [400/600], Loss: 0.0000
Epoch [2/30], Step [500/600], Loss: 0.0000
Epoch [2/30], Step [600/600], Loss: 0.0000
Epoch [3/30], Step [100/600], Loss: 0.0000
Epoch [3/30], Step [200/600], Loss: 0.0000
Epoch [3/30], Step [300/600], Loss: 0.0000
Epoch [3/30], Step [400/600], Loss: 0.0000
Epoch [3/30], Step [500/600], Loss: 0.0000
Epoch [3/30], Step [600/600], Loss: 0.0000
Epoch [4/30], Step [100/600], Loss: 0.0000
Epoch [4/30], Step [200/600], Loss: 0.0000
Epoch [4/30], Step [300/600], Loss: 0.0002
Epoch [4/30], Step [400/600], Loss: 0.0149
Epoch [4/30], Step [500/600], Loss: 0.0000
Epoch [4/30

In [36]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the 10000 test images:{} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images:99.25 %


In [32]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model_PyTorch.ckpt')